In [1]:
import websockets
import json
import pandas as pd
import asyncio
import threading
from colorama import Fore, Back, Style
import timeit
from tkinter import *
start = timeit.default_timer()

In [2]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','c','v']]
    df.columns = ['Symbol','Time','Price','Volume']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [3]:
def get_sma(l, p):
    return sum(l[-p:])/p

In [4]:
coins = ['dogeusdt','btcusdt','ethusdt','linkusdt','uniusdt','algousdt','maticusdt','compusdt','lunausdt','dodousdt','ankrusdt','milusdt','gtcusdt','ampusdt','aaveusdt','adausdt','cakeusdt','bnbusdt','crvusdt','xlmusdt','sushiusdt','grtusdt']

async def get_data(pair):
    first_time = True
    vol_sig = False
    price_sig = False
    
    stream = websockets.connect(f'wss://stream.binance.com:9443/stream?streams={pair}@miniTicker')
    while(True):
        async with stream as receiver:   
            data = await receiver.recv()
            data = json.loads(data)['data']
            if first_time == True:
                df = createframe(data)
                first_time = False
                account = {
                    "Symbol": df.iloc[0]['Symbol'],
                    "Balance": 1000,
                    "Holdings": 0
                }
            else:
                df = df.append(createframe(data), ignore_index=True)
            
            if len(df.index) > ma_points:
                ma = get_sma(df["Price"], ma_points)
                curr_price = float(df.iloc[-1]["Price"])
                prev_price = float(df.iloc[-2]["Price"])
                curr_vol = float(df.iloc[-1]["Volume"])
                prev_vol = float(df.iloc[-2]["Volume"])
                
                time = pd.to_datetime(df.iloc[-1]["Time"], unit="ms")
                if curr_price > prev_price + (prev_price * price_percentage_change):
                    price_sig = True
#                     print(Fore.GREEN)
#                     print(f"{time}: {df.iloc[0]['Symbol']} rose from {prev_price} to {curr_price}")
#                     print(Style.RESET_ALL)
                if curr_vol > prev_vol + (prev_vol * volume_percentage_change):
                    vol_sig = True
                    print(f"{time}: volume of {df.iloc[0]['Symbol']} rose by at least {volume_percentage_change * 100}%")

                if curr_price > ma and price_sig and vol_sig:
                    print(Fore.GREEN)
                    stocks = 100/curr_price
                    account["Holdings"] = account["Holdings"] + stocks
                    account["Balance"] = account["Balance"] - 100
                    print(f"Bought {stocks} units of {df.iloc[0]['Symbol']} at {curr_price}")
                    print(account)
                    print("Total bag: ", account["Holdings"] * curr_price + account["Balance"])
                    print(Style.RESET_ALL)
                    price_sig = False
                    vol_sig = False
                    print(f"{timeit.default_timer() - start} seconds elapsed")
                    
                if curr_price < ma and account["Holdings"] != 0:
                    print(Fore.RED)
                    print(f"Sold {account['Holdings']} units of {df.iloc[0]['Symbol']} at {curr_price}")
                    account["Balance"] = account["Balance"] + account["Holdings"] * curr_price
                    account["Holdings"] = 0
                    print(account)
                    print("Total monies: ", account["Balance"])
                    print(Style.RESET_ALL)
                    print(f"{timeit.default_timer() - start} seconds elapsed")
                      
            await asyncio.sleep(interval)
        
def wrapper_func(pair):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    loop.run_until_complete(get_data(pair))
    loop.close()

In [5]:
def run_program():
    global price_percentage_change 
    global volume_percentage_change 
    global interval
    global ma_points

    interval = int(interval_field.get())
    price_percentage_change = float(ppc_field.get()) / 100
    volume_percentage_change = float(pvc_field.get()) / 100
    ma_points = int(ma_field.get())
    
    def master_thread():
        threads = []
        for coin in coins:
            print(f"Starting thread for {coin}")
            t = threading.Thread(target=wrapper_func, args=(coin, ))
            threads.append(t)
            t.start()

        for thread in threads:
            thread.join()
            
    mt = threading.Thread(target=master_thread)
    mt.start()

In [ ]:
root = Tk()

root.title("Crypto Trader")
interval_label = Label(root, text='Interval (in seconds):', padx=5)
interval_label.grid(column = 0, row = 1)
interval_field = Entry(root, width = 20)
interval_field.grid(column =1, row = 1)

ppc_label = Label(root, text='Price percentage change:')
ppc_label.grid(column = 0, row = 2)
ppc_field = Entry(root, width = 20)
ppc_field.grid(column =1, row = 2)

pvc_label = Label(root, text='Volume percentage change:')
pvc_label.grid(column = 0, row = 3)
pvc_field = Entry(root, width = 20)
pvc_field.grid(column =1, row = 3)

ma_label = Label(root, text='Moving average points')
ma_label.grid(column = 0, row = 4)
ma_field = Entry(root, width = 20)
ma_field.grid(column =1, row = 4)


button = Button(root, text='Get Money', command = run_program)
button.grid(column = 1, row = 5)


root.mainloop()

Starting thread for dogeusdt
Starting thread for btcusdt
Starting thread for ethusdt
Starting thread for linkusdt
Starting thread for uniusdt
Starting thread for algousdt
Starting thread for maticusdt
Starting thread for compusdt
Starting thread for lunausdt
Starting thread for dodousdt
Starting thread for ankrusdt
Starting thread for milusdt
Starting thread for gtcusdt
Starting thread for ampusdt
Starting thread for aaveusdt
Starting thread for adausdt
Starting thread for cakeusdt
Starting thread for bnbusdt
Starting thread for crvusdt
Starting thread for xlmusdt
Starting thread for sushiusdt
Starting thread for grtusdt
2022-02-01 07:45:21.599000: volume of BTCUSDT rose by at least 0.02%
2022-02-01 07:45:23.244000: volume of ALGOUSDT rose by at least 0.02%
2022-02-01 07:45:29.423000: volume of DOGEUSDT rose by at least 0.02%
2022-02-01 07:45:32.643000: volume of AAVEUSDT rose by at least 0.02%
2022-02-01 07:45:42.643000: volume of SUSHIUSDT rose by at least 0.02%
2022-02-01 07:45:52.0

# 